<a href="https://colab.research.google.com/github/MurtazaPakawala/insurance_cost_predictor/blob/main/insurance_cost_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torch.nn as nn
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split


downloading and playing with data


In [ ]:
DATASET_URL = "https://gist.github.com/BirajCoder/5f068dfe759c1ea6bdfce9535acdb72d/raw/c84d84e3c80f93be67f6c069cbdc0195ec36acbd/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

  0%|          | 0/54288 [00:00<?, ?it/s]

In [ ]:
insurance_df = pd.read_csv(DATA_FILENAME)

In [ ]:
insurance_df.shape

(1338, 7)

In [ ]:
row_nums=insurance_df.shape[0]
col_nums=insurance_df.shape[1]

In [ ]:
insurance_df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:
insurance_df.charges.describe()

count     1338.000000
mean     13270.422265
std      12110.011237
min       1121.873900
25%       4740.287150
50%       9382.033000
75%      16639.912515
max      63770.428010
Name: charges, dtype: float64

converting the dataframe into tensors

In [ ]:
input_cols = ["age", "sex","bmi","children","smoker","region"]
categorical_cols = ["sex","smoker","region"]
output_cols = ["charges"]

In [ ]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

In [ ]:
input_array,target_array=dataframe_to_arrays(insurance_df)
input_array,target_array

(array([[19.  ,  0.  , 27.9 ,  0.  ,  1.  ,  3.  ],
        [18.  ,  1.  , 33.77,  1.  ,  0.  ,  2.  ],
        [28.  ,  1.  , 33.  ,  3.  ,  0.  ,  2.  ],
        ...,
        [18.  ,  0.  , 36.85,  0.  ,  0.  ,  2.  ],
        [21.  ,  0.  , 25.8 ,  0.  ,  0.  ,  3.  ],
        [61.  ,  0.  , 29.07,  0.  ,  1.  ,  1.  ]]), array([[16884.924 ],
        [ 1725.5523],
        [ 4449.462 ],
        ...,
        [ 1629.8335],
        [ 2007.945 ],
        [29141.3603]]))

In [ ]:
inputs = torch.tensor(input_array,dtype=torch.float32)
targets = torch.tensor(target_array, dtype = torch.float32)

In [ ]:
print(type(inputs),inputs.dtype)

<class 'torch.Tensor'> torch.float32


In [ ]:
dataset = TensorDataset(inputs,target)

In [ ]:
val_percent = 0.2
val_size = int(0.2*row_nums)
print("the validation size is : ",val_size)
train_size = row_nums-val_size
print("the training size is : ",train_size)

the validation size is :  267
the training size is :  1071


In [ ]:
train_ds, val_ds = random_split(dataset,[train_size,val_size])

making the data loader and splitting in batch size of 32

In [ ]:
batch_size=32
train_loader = DataLoader(train_ds,batch_size,shuffle=True)
val_loader = DataLoader(val_ds,batch_size,shuffle=True)

In [ ]:
for xb,yb in train_loader:
  print(xb)
  print(yb)
  break

tensor([[38.0000,  1.0000, 21.1200,  3.0000,  0.0000,  2.0000],
        [64.0000,  1.0000, 33.8800,  0.0000,  1.0000,  2.0000],
        [18.0000,  1.0000, 23.7500,  0.0000,  0.0000,  0.0000],
        [45.0000,  1.0000, 24.0350,  2.0000,  0.0000,  0.0000],
        [19.0000,  1.0000, 26.0300,  1.0000,  1.0000,  1.0000],
        [32.0000,  0.0000, 17.7650,  2.0000,  1.0000,  1.0000],
        [53.0000,  0.0000, 33.2500,  0.0000,  0.0000,  0.0000],
        [54.0000,  1.0000, 39.6000,  1.0000,  0.0000,  3.0000],
        [50.0000,  0.0000, 27.0750,  1.0000,  0.0000,  0.0000],
        [21.0000,  1.0000, 36.8600,  0.0000,  0.0000,  1.0000],
        [23.0000,  0.0000, 28.0000,  0.0000,  0.0000,  3.0000],
        [22.0000,  0.0000, 36.0000,  0.0000,  0.0000,  3.0000],
        [19.0000,  1.0000, 34.1000,  0.0000,  0.0000,  3.0000],
        [44.0000,  1.0000, 37.1000,  2.0000,  0.0000,  3.0000],
        [18.0000,  1.0000, 22.9900,  0.0000,  0.0000,  0.0000],
        [39.0000,  1.0000, 26.4100,  0.0